In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

import statistics
import csv
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

from warnings import simplefilter

2023-10-04 15:43:30.821907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# -- Input

train_data_set_code = "normal_1_14"
fpr_test_data_set_code = "fpr-validation"

test_data_set_codes_general = ["cpu-stress", "mem-leak", "pack-loss", "pack-delay", "pack-corr"]
replicas_num = 3


if replicas_num > 1:
    test_data_set_codes = ["{code}-{replica}".format(code=ii, replica=jj) for ii in test_data_set_codes_general for jj in range(replicas_num)]
else:
    test_data_set_codes = test_data_set_codes_general

test_data_set_codes = test_data_set_codes + [fpr_test_data_set_code]
    
# Set of lists with the experiments schedules - [whole duration, fault injection point, failre point]
data_sets_config = {"normal_test_24h": [1440, 0, 0], "fpr-validation": [10080, 0, 0],
                    "cpu-stress-0": [45, 16, 42], "mem-leak-0": [60, 16, 51], "pack-loss-0": [30, 16, 25], "pack-delay-0": [30, 16, 25], "pack-corr-0": [30, 16, 24],
                    "cpu-stress-1": [45, 16, 40], "mem-leak-1": [60, 16, 43], "pack-loss-1": [30, 16, 22], "pack-delay-1": [30, 16, 24], "pack-corr-1": [30, 16, 28],
                    "cpu-stress-2": [45, 16, 42], "mem-leak-2": [60, 16, 49], "pack-loss-2": [30, 16, 23], "pack-delay-2": [30, 16, 23], "pack-corr-2": [30, 16, 25]}


# -- Paths

# Path to the raw datasets
source_dir_path = "resources/datasets/{data_set_code}_consolidated/"
source_file_path = source_dir_path + "{data_set_code}.csv"

# Path to the preprocessed datasets
target_dir_path = "resources/datasets/{data_set_code}_tuned/"
target_file_path = target_dir_path + "{data_set_code}.csv"

target_dir_path_rbm = "resources/datasets/{data_set_code}_rbm/"
target_file_path_rbm = target_dir_path_rbm + "{data_set_code}.csv"

final_feature_list_file_path = 'resources/features/features_final.csv'


# -- Init the Silencer
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


# -- Functions

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(abs(value))
    return diff

# invert differenced forecast
def inverse_difference(last_ob, value):
    return value + last_ob

def difference_order(dataset, interval=1, order=1):
    for ii in range(order):
        dataset = [0] + difference(dataset, interval)

    return dataset

def get_predictions(loss_, threshold_):
    return tf.math.greater(loss_, threshold_)

def print_stats(predictions_, labels_):
    print("\n")
    print("Accuracy = {}".format(accuracy_score(labels_, predictions_)))
    print("Precision = {}".format(precision_score(labels_, predictions_)))
    print("Recall = {}".format(recall_score(labels_, predictions_)))

def plot_samples(data_, minute_of_experiment_, title_):
    plt.grid()
    plt.plot(np.arange(len(data_[minute_of_experiment_])), data_[minute_of_experiment_][:])
    plt.title(title_)
    plt.show()

# Print a loss distribution (reconstruction error / number of examples)
def plot_loss_distribution(loss_, title_, color_="skyblue"):
    plt.figure(figsize=(10, 5))
    plt.hist(loss_[None,:], bins=50, color=color_)
    plt.xlabel("Loss (reconstruction error)")
    plt.ylabel("Number of points")
    plt.title(title_)
    plt.show()

# Threshold value = one standard deviations above the mean
def get_threshold(loss_):
    threshold_ = np.mean(loss_) + np.std(loss_)
    print("Mean:", np.mean(loss_),"Std Deviation:", np.std(loss_),  "Threshold: ", threshold_)
    return threshold_

def visualise_data_set_dynamics(df_, data_set_code_, predictions_, data_sets_config_):
    
    df_ = df_.copy()
    
    point_fault_injection = data_sets_config_[data_set_code_][1]
    point_failure = data_sets_config_[data_set_code_][2]

    fig = px.line(df_, y=df_.columns, x=np.arange(len(df_.values)), title=str(data_set_code_), width=2000, height=1200)

    fig.add_vrect(x0=point_fault_injection, x1=point_failure,
                  annotation_text="Fault injected", annotation_position="top left",
                  fillcolor="blue", opacity=0.25, line_width=0)

    predictions = list(map(int, predictions_))
    for ii in range(len(predictions)):
        if predictions[ii] == 1:
            fig.add_vrect(x0=ii, x1=ii, line_color="red", opacity=0.25)

    # fig.show()
    fig.write_html("resources/data_set_dynamics/{data_set_code}.html".format(data_set_code=data_set_code_))


def stationarity_test(series_):
    X = series_.values
    result = adfuller(X)

    # print('ADF Statistic: %f' % result[0])
    # print('p-value: %f' % result[1])
    # print('Critical Values:')

    return result[0], result[1]


def outl(series):
    mean_ = np.median(series)
    std_ = np.std(series)

    up = mean_ + std_ * 5
    bottom = mean_ - std_ * 5

    for xx in range(len(series)):
        if series[xx] > up:
            print("up", series[xx], up)
            series[xx] = up
        else:
            if series[xx] < bottom:
                print("bottom", series[xx], bottom)
                series[xx] = bottom

    return series

In [3]:
# -- Load the training dataset
df = pd.read_csv(source_file_path.format(data_set_code=train_data_set_code))

df.head()

,timestamp,redis-1-1_system.network.in.bytes,redis-1-1_system.network.in.dropped,redis-1-1_system.network.in.packets,redis-1-1_system.network.in.errors,redis-1-1_system.network.name,redis-1-1_system.network.out.bytes,redis-1-1_system.network.out.dropped,redis-1-1_system.network.out.errors,redis-1-1_system.network.out.packets,...,redis-1-9_system.socket.summary.all.count,redis-1-9_system.socket.summary.tcp.all.established,redis-1-9_system.socket.summary.tcp.all.listening,redis-1-9_system.socket.summary.tcp.all.time_wait,redis-1-9_system.socket.summary.tcp.all.count,redis-1-9_system.socket.summary.tcp.all.close_wait,redis-1-9_system.socket.summary.tcp.all.orphan,redis-1-9_system.socket.summary.tcp.memory,redis-1-9_system.socket.summary.udp.all.count,redis-1-9_system.socket.summary.udp.memory
0,2022-11-07T17:30:00.000Z,1130921095,0,4682651,0,ens4,1064572240,0,0,5109595,...,86,74,7,1,82,0,0,126976,4,4096
1,2022-11-07T17:31:00.000Z,1133969677,0,4703090,0,ens4,1067519274,0,0,5133289,...,85,74,7,0,81,0,0,126976,4,4096
2,2022-11-07T17:32:00.000Z,1136941742,0,4722891,0,ens4,1070362384,0,0,5156508,...,86,74,7,1,82,0,0,126976,4,4096
3,2022-11-07T17:33:00.000Z,1139992399,0,4743106,0,ens4,1073286364,0,0,5179975,...,85,74,7,0,81,0,0,126976,4,4096
4,2022-11-07T17:34:00.000Z,1142967381,0,4762924,0,ens4,1076154281,0,0,5203291,...,86,74,7,1,82,0,0,126976,4,4096


In [4]:
# -- Drop the timestamp column

timestamp_values_list = df["timestamp"].tolist()
df.drop('timestamp', axis=1, inplace=True)

# Keep only numeric types
df = df.replace('ens4', 1)

# Convert to float
df = df.astype('float')

df.head()

,redis-1-1_system.network.in.bytes,redis-1-1_system.network.in.dropped,redis-1-1_system.network.in.packets,redis-1-1_system.network.in.errors,redis-1-1_system.network.name,redis-1-1_system.network.out.bytes,redis-1-1_system.network.out.dropped,redis-1-1_system.network.out.errors,redis-1-1_system.network.out.packets,redis-1-10_system.network.in.bytes,...,redis-1-9_system.socket.summary.all.count,redis-1-9_system.socket.summary.tcp.all.established,redis-1-9_system.socket.summary.tcp.all.listening,redis-1-9_system.socket.summary.tcp.all.time_wait,redis-1-9_system.socket.summary.tcp.all.count,redis-1-9_system.socket.summary.tcp.all.close_wait,redis-1-9_system.socket.summary.tcp.all.orphan,redis-1-9_system.socket.summary.tcp.memory,redis-1-9_system.socket.summary.udp.all.count,redis-1-9_system.socket.summary.udp.memory
0,1.130921e+09,0.0,4682651.0,0.0,1.0,1.064572e+09,0.0,0.0,5109595.0,1.065564e+09,...,86.0,74.0,7.0,1.0,82.0,0.0,0.0,126976.0,4.0,4096.0
1,1.133970e+09,0.0,4703090.0,0.0,1.0,1.067519e+09,0.0,0.0,5133289.0,1.068593e+09,...,85.0,74.0,7.0,0.0,81.0,0.0,0.0,126976.0,4.0,4096.0
2,1.136942e+09,0.0,4722891.0,0.0,1.0,1.070362e+09,0.0,0.0,5156508.0,1.071631e+09,...,86.0,74.0,7.0,1.0,82.0,0.0,0.0,126976.0,4.0,4096.0
3,1.139992e+09,0.0,4743106.0,0.0,1.0,1.073286e+09,0.0,0.0,5179975.0,1.074667e+09,...,85.0,74.0,7.0,0.0,81.0,0.0,0.0,126976.0,4.0,4096.0
4,1.142967e+09,0.0,4762924.0,0.0,1.0,1.076154e+09,0.0,0.0,5203291.0,1.077715e+09,...,86.0,74.0,7.0,1.0,82.0,0.0,0.0,126976.0,4.0,4096.0


In [5]:
# -- Drop the KPIs which demonstrate the anomalous behaviour even during the normal execution (before fault injection)

df = df.drop(df.filter(like='fsstat', axis=1), axis=1)
df = df.drop(df.filter(like='memory.total', axis=1), axis=1)
df = df.drop(df.filter(like='memory.used', axis=1), axis=1)
df = df.drop(df.filter(like='memory.free', axis=1), axis=1)

df.head()

,redis-1-1_system.network.in.bytes,redis-1-1_system.network.in.dropped,redis-1-1_system.network.in.packets,redis-1-1_system.network.in.errors,redis-1-1_system.network.name,redis-1-1_system.network.out.bytes,redis-1-1_system.network.out.dropped,redis-1-1_system.network.out.errors,redis-1-1_system.network.out.packets,redis-1-10_system.network.in.bytes,...,redis-1-9_system.socket.summary.all.count,redis-1-9_system.socket.summary.tcp.all.established,redis-1-9_system.socket.summary.tcp.all.listening,redis-1-9_system.socket.summary.tcp.all.time_wait,redis-1-9_system.socket.summary.tcp.all.count,redis-1-9_system.socket.summary.tcp.all.close_wait,redis-1-9_system.socket.summary.tcp.all.orphan,redis-1-9_system.socket.summary.tcp.memory,redis-1-9_system.socket.summary.udp.all.count,redis-1-9_system.socket.summary.udp.memory
0,1.130921e+09,0.0,4682651.0,0.0,1.0,1.064572e+09,0.0,0.0,5109595.0,1.065564e+09,...,86.0,74.0,7.0,1.0,82.0,0.0,0.0,126976.0,4.0,4096.0
1,1.133970e+09,0.0,4703090.0,0.0,1.0,1.067519e+09,0.0,0.0,5133289.0,1.068593e+09,...,85.0,74.0,7.0,0.0,81.0,0.0,0.0,126976.0,4.0,4096.0
2,1.136942e+09,0.0,4722891.0,0.0,1.0,1.070362e+09,0.0,0.0,5156508.0,1.071631e+09,...,86.0,74.0,7.0,1.0,82.0,0.0,0.0,126976.0,4.0,4096.0
3,1.139992e+09,0.0,4743106.0,0.0,1.0,1.073286e+09,0.0,0.0,5179975.0,1.074667e+09,...,85.0,74.0,7.0,0.0,81.0,0.0,0.0,126976.0,4.0,4096.0
4,1.142967e+09,0.0,4762924.0,0.0,1.0,1.076154e+09,0.0,0.0,5203291.0,1.077715e+09,...,86.0,74.0,7.0,1.0,82.0,0.0,0.0,126976.0,4.0,4096.0


In [6]:
# -- Remove Constant Features using Variance Threshold
constant_filter = VarianceThreshold(threshold=0.00001)
constant_filter.fit(df)
constant_columns = [column for column in df.columns if column not in df.columns[constant_filter.get_support()]]
df.drop(labels=constant_columns, axis=1, inplace=True)

df.head()

,redis-1-1_system.network.in.bytes,redis-1-1_system.network.in.packets,redis-1-1_system.network.out.bytes,redis-1-1_system.network.out.packets,redis-1-10_system.network.in.bytes,redis-1-10_system.network.in.packets,redis-1-10_system.network.out.bytes,redis-1-10_system.network.out.packets,redis-1-11_system.network.in.bytes,redis-1-11_system.network.in.packets,...,redis-1-8_system.socket.summary.tcp.all.count,redis-1-8_system.socket.summary.tcp.all.orphan,redis-1-8_system.socket.summary.tcp.memory,redis-1-9_system.socket.summary.all.count,redis-1-9_system.socket.summary.tcp.all.established,redis-1-9_system.socket.summary.tcp.all.time_wait,redis-1-9_system.socket.summary.tcp.all.count,redis-1-9_system.socket.summary.tcp.all.orphan,redis-1-9_system.socket.summary.tcp.memory,redis-1-9_system.socket.summary.udp.memory
0,1.130921e+09,4682651.0,1.064572e+09,5109595.0,1.065564e+09,4627493.0,1.059335e+09,5068571.0,965299538.0,2172184.0,...,82.0,0.0,126976.0,86.0,74.0,1.0,82.0,0.0,126976.0,4096.0
1,1.133970e+09,4703090.0,1.067519e+09,5133289.0,1.068593e+09,4647588.0,1.062275e+09,5092331.0,967062712.0,2180513.0,...,81.0,0.0,131072.0,85.0,74.0,0.0,81.0,0.0,126976.0,4096.0
2,1.136942e+09,4722891.0,1.070362e+09,5156508.0,1.071631e+09,4667692.0,1.065241e+09,5116258.0,968880136.0,2188751.0,...,82.0,0.0,126976.0,86.0,74.0,1.0,82.0,0.0,126976.0,4096.0
3,1.139992e+09,4743106.0,1.073286e+09,5179975.0,1.074667e+09,4687698.0,1.068189e+09,5139947.0,970649445.0,2196933.0,...,81.0,0.0,131072.0,85.0,74.0,0.0,81.0,0.0,126976.0,4096.0
4,1.142967e+09,4762924.0,1.076154e+09,5203291.0,1.077715e+09,4707791.0,1.071168e+09,5163868.0,972430836.0,2205216.0,...,82.0,0.0,126976.0,86.0,74.0,1.0,82.0,0.0,126976.0,4096.0


In [7]:
# -- Save the training dataset

# Create a target folder is does not exist
if not os.path.exists(target_dir_path.format(data_set_code=train_data_set_code)):
    os.makedirs(target_dir_path.format(data_set_code=train_data_set_code))

df.to_csv(target_file_path.format(data_set_code=train_data_set_code), encoding='utf-8', index=False, header=True)

In [8]:
# -- Save the training dataset copy for Prebvent-E

# Copy and insert a timestamp column
df_prevent_e = df.copy()
df_prevent_e.insert(loc=0, column='timestamp', value=timestamp_values_list)

# Create a target folder is does not exist
if not os.path.exists(target_dir_path_rbm.format(data_set_code=train_data_set_code)):
    os.makedirs(target_dir_path_rbm.format(data_set_code=train_data_set_code))

# Save the dataset
df_prevent_e.to_csv(target_file_path_rbm.format(data_set_code=train_data_set_code), encoding='utf-8', index=False, header=True)

In [9]:
# -- Save the names of the final columns

df_tmp = pd.DataFrame(df.columns)
df_tmp.to_csv(final_feature_list_file_path, encoding='utf-8', index=False, header=None)

In [10]:
# -- Get the KPIs of the final training set

df_tmp = pd.read_csv(final_feature_list_file_path, sep=',', header=None)
features_to_keep = df_tmp[0].values

In [11]:
# -- Preprocess the expetimental data sets

# Loop by test data-sets
for test_data_set_code in test_data_set_codes:

    # Read to the data-frame
    df = pd.read_csv(source_file_path.format(data_set_code=test_data_set_code), index_col=False)

    # Keep only the training KPIs
    timestamp_values_list = df["timestamp"].tolist()
    df = df[features_to_keep]
    
    # Keep only numeric types
    df = df.replace('ens4', 1)

    # Convert to float
    df = df.astype('float')

    # Create the target folder is does not exist
    if not os.path.exists(target_dir_path.format(data_set_code=test_data_set_code)):
        os.makedirs(target_dir_path.format(data_set_code=test_data_set_code))

    # Save the dataset
    df.to_csv(target_file_path.format(data_set_code=test_data_set_code), encoding='utf-8', index=False, header=True)
    
    
    # -- Save the dataset copy for Prebvent-E

    # Copy and insert a timestamp column
    df_prevent_e = df.copy()
    df_prevent_e.insert(loc=0, column='timestamp', value=timestamp_values_list)

    # Create a targetv folder is does not exist
    if not os.path.exists(target_dir_path_rbm.format(data_set_code=test_data_set_code)):
        os.makedirs(target_dir_path_rbm.format(data_set_code=test_data_set_code))

    # Save the dataset
    df_prevent_e.to_csv(target_file_path_rbm.format(data_set_code=test_data_set_code), encoding='utf-8', index=False, header=True)